<a href="https://colab.research.google.com/github/araujoghm/DataScienceEMAp_AraujoNovais/blob/master/FDS_part2_stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Airbnb listing prices x nearby crimes on each date 

## Statistical analysis

As a preliminary step, we'll check correlations between price and our counts of nearby crimes on the date of the listing.

In [0]:
Z_c=airbnb_cr[['price','crimes_1km_count','crimes_2km_count','crimes_5km_count']]
Z_h=airbnb_cr[['price','homicides_1km_count','homicides_2km_count','homicides_5km_count']]
Z_v=airbnb_cr[['price','violence_1km_count','violence_2km_count','violence_5km_count']]
Z_s=airbnb_cr[['price','stealing_1km_count','stealing_2km_count','stealing_5km_count']]

In [0]:
Z_h.corr()

,price,homicides_1km_count,homicides_2km_count,homicides_5km_count
price,1.000000,-0.012954,-0.035319,-0.074769
homicides_1km_count,-0.012954,1.000000,0.021951,0.025620
homicides_2km_count,-0.035319,0.021951,1.000000,0.118846
homicides_5km_count,-0.074769,0.025620,0.118846,1.000000


In [0]:
Z_v.corr()

,price,violence_1km_count,violence_2km_count,violence_5km_count
price,1.000000,0.086223,0.057213,0.082182
violence_1km_count,0.086223,1.000000,0.370663,0.330300
violence_2km_count,0.057213,0.370663,1.000000,0.530205
violence_5km_count,0.082182,0.330300,0.530205,1.000000


In [0]:
Z_s.corr()

,price,stealing_1km_count,stealing_2km_count,stealing_5km_count
price,1.000000,0.082256,0.003433,-0.002306
stealing_1km_count,0.082256,1.000000,0.295942,0.095814
stealing_2km_count,0.003433,0.295942,1.000000,0.330896
stealing_5km_count,-0.002306,0.095814,0.330896,1.000000


Correlations above are very close to 0, which suggests price and occurences of those types of crimes are not much correlated. With the exception of homicides, crimes still seem to have a very weak but positive correlation with prices. 

Given we have left out so many important variables, we'll have better insight after doing a regression, which will take into account factors such as the type of room (entire home/apt, private room or shared room), neighborhood, listing availability, the listing history (number of reviews, reviews per month), as well as including counts of each of our categories of crime for our 3 radiuses. Note that even when take this into account, we still have a major omitted variable problem, since we don't have any kind of socioeconomical and demographical information other than what's encapsulated by the neighborhood dummy variable, but as said before, this is a rough estimate that's supposed to hint to us whether there's an evidence that nearby criminal occurences influence the supply-demand model for Airbnb listings, particularly on the city of Chicago.

We'll set up our regression by selecting which variables to include, as well as creating dummies for neighbourhoods and room types. Below, we'll check the results of our regression.

In [0]:
X = airbnb_cr[['crimes_1km_count','crimes_2km_count','crimes_5km_count','crimes_count_date',
               'homicides_1km_count','homicides_2km_count','homicides_5km_count','homicides_count_date',
               'violence_1km_count','violence_2km_count','violence_5km_count','violence_count_date',
               'stealing_1km_count','stealing_2km_count','stealing_5km_count','stealing_count_date',
               'room_type','neighbourhood','number_of_reviews','reviews_per_month','availability_365']]
y = airbnb_cr[['price']]

X = pd.concat([X, pd.get_dummies(X.room_type, prefix='room_type', drop_first=True)], axis=1)
X = pd.concat([X, pd.get_dummies(X.neighbourhood, drop_first=True)], axis=1)

X=X.drop(['neighbourhood','room_type'],axis=1)

reg = LinearRegression().fit(X, y)


X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.292
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     448.6
Date:                Wed, 04 Sep 2019   Prob (F-statistic):               0.00
Time:                        17:43:47   Log-Likelihood:            -5.8394e+05
No. Observations:               97060   AIC:                         1.168e+06
Df Residuals:                   96970   BIC:                         1.169e+06
Df Model:                          89                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     30

For further explanation and commentary on the results, check the Report section of our website.

##Validating results

#2 - Airbnb listed price changes x criminal occurences variation

##Statistical analysis

Like before, we'll do a regression but this time we'll considere the difference between scraping dates in regards to price and crime counts. Note that this model implies a very strong (and rough) assumption: that the influence crime had on the price can be measured by the difference in criminal occurences between scraping dates, and nothing more. Again, this is not a very precise estimate but should be enough to suggest us whether there's an influence at all.

In [0]:
X = cal_change_r[['crimes_1km_count_dif','crimes_2km_count_dif','crimes_5km_count_dif',
               'homicides_1km_count_dif','homicides_2km_count_dif','homicides_5km_count_dif',
               'violence_1km_count_dif','violence_2km_count_dif','violence_5km_count_dif',
               'stealing_1km_count_dif','stealing_2km_count_dif','stealing_5km_count_dif',
               'room_type','neighbourhood','number_of_reviews','reviews_per_month','availability_365']]
y = cal_change_r[['price_dif']]

X = pd.concat([X, pd.get_dummies(X.room_type, prefix='room_type', drop_first=True)], axis=1)
X = pd.concat([X, pd.get_dummies(X.neighbourhood, drop_first=True)], axis=1)

X=X.drop(['neighbourhood','room_type'],axis=1)

reg = LinearRegression().fit(X, y)

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


                            OLS Regression Results                            
Dep. Variable:              price_dif   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     70.03
Date:                Wed, 04 Sep 2019   Prob (F-statistic):               0.00
Time:                        23:36:16   Log-Likelihood:            -3.6202e+05
No. Observations:               59947   AIC:                         7.242e+05
Df Residuals:                   59862   BIC:                         7.250e+05
Df Model:                          84                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

For an interpretation of the results, check the Report section of our website.

## Validating results